### DATA305 Assignment 1: Problem 4.
- Name: Armand Faris A Surbakti
- Student ID: 300680133

In [2]:
import numpy as np
import pandas as pd
import os
import re
from random import shuffle
from majority_voter import majority_voter
from dtree import dtree
from randomized_voter import randomized_majority_voter
from randomforest import randomforest
from collections import Counter
from bagging import bagger

In [3]:
df = pd.read_csv("breast-cancer.data", header=None)
df

,0,1,2,3,4,5,6,7,8,9
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no
...,...,...,...,...,...,...,...,...,...,...
281,recurrence-events,30-39,premeno,30-34,0-2,no,2,left,left_up,no
282,recurrence-events,30-39,premeno,20-24,0-2,no,3,left,left_up,yes
283,recurrence-events,60-69,ge40,20-24,0-2,no,1,right,left_up,no
284,recurrence-events,40-49,ge40,30-34,3-5,no,3,left,left_low,no


In [4]:
#initialize all the classes
tree = dtree()
bagger = bagger()
forest = randomforest()
voter = majority_voter()
random_voter = randomized_majority_voter()

data, classes, features = tree.read_data("breast-cancer.data")

In [5]:
#split data into training and testing sets
train = data[::2]
test = data[1::2]
trainc = classes[::2]
testc = classes[1::2]

which_gain = 'info'

In [6]:
#test and train using the decision tree
t = tree.make_tree(train, trainc, features, which_gain)
out = tree.classifyAll(t, test)

In [7]:
a = np.zeros(len(out))
b = np.zeros(len(out))
d = np.zeros(len(out))


for i in range(len(out)):
    if testc[i] in ['good', 'v-good']:
        b[i] = 1
        if out[i] == testc[i]:
            d[i] = 1
    if out[i] == testc[i]:
        a[i] = 1


print("Decision Tree Training")
print("Tree")
print("Number correctly predicted", np.sum(a))
print("Number of test points", len(a))
print("Percentage Accuracy", np.sum(a) / len(a) * 100.0)

Decision Tree Training
Tree
Number correctly predicted 87.0
Number of test points 142
Percentage Accuracy 61.267605633802816


Training using the Bagger

In [8]:
c = bagger.bag(train, trainc, features, which_gain, 100)
out = bagger.bagclass(c, test)

a = np.zeros(len(out))
b = np.zeros(len(out))
d = np.zeros(len(out))

for i in range(len(out)):
    if testc[i] in ['good', 'v-good']:
        b[i] = 1
        if out[i] == testc[i]:
            d[i] = 1
    if out[i] == testc[i]:
        a[i] = 1

print("-----")
print("Bagger Training")
print("Number correctly predicted", np.sum(a))
print("Number of test points", len(a))
print("Percentage Accuracy", np.sum(a) / len(a) * 100.0)


-----
Bagger Training
Number correctly predicted 110.0
Number of test points 142
Percentage Accuracy 77.46478873239437


Train using the Random Forest Model

In [9]:
f = forest.rf(train, trainc, features, which_gain, 100, 200, 2, maxlevel=3)
out = forest.rfclass(f, test)

a = np.zeros(len(out))
b = np.zeros(len(out))
d = np.zeros(len(out))

for i in range(len(out)):
    if testc[i] in ['good', 'v-good']:
        b[i] = 1
        if out[i] == testc[i]:
            d[i] = 1
    if out[i] == testc[i]:
        a[i] = 1


print("Random Forest Training")
print("Number correctly predicted", np.sum(a))
print("Number of test points", len(a))
print("Percentage Accuracy", np.sum(a) / len(a) * 100.0)

Random Forest Training
Number correctly predicted 107.0
Number of test points 142
Percentage Accuracy 75.35211267605634


Weighted Majority Voting

In [10]:
mv = voter.make_classifiers(train, trainc, features, which_gain, 100, 200, 2, maxlevel=5)
out = voter.vote(mv, test, testc)

#calculate accuracy
a = np.zeros(len(out))
b = np.zeros(len(out))
d = np.zeros(len(out))

for i in range(len(out)):
    if testc[i] in ['good', 'v-good']:
        b[i] = 1
        if out[i] == testc[i]:
            d[i] = 1
    if out[i] == testc[i]:
        a[i] = 1
    
print("Weighted Majority Voter Training")
print("Number correctly predicted", np.sum(a))
print("Number of test points", len(a))
print("Percentage Accuracy", np.sum(a) / len(a) * 100.0)

Updated weights: [2.65173413e+12 4.38548343e+14 4.38548343e+14 3.41015192e+13
 2.61100361e+16 5.63976779e+15 7.30913905e+14 4.38548343e+14
 3.41015192e+13 4.38548343e+14 9.39961298e+15 1.56660216e+16
 9.47264421e+13 3.41015192e+13 2.04609115e+13 3.41015192e+13
 3.41015192e+13 4.38548343e+14 4.38548343e+14 4.38548343e+14
 7.36592814e+12 2.63129006e+14 3.41015192e+13 7.36592814e+12
 1.22765469e+13 3.38386067e+15 1.21818984e+15 7.30913905e+14
 2.03031640e+15 9.39961298e+15 5.68358653e+13 2.03031640e+15
 3.41015192e+13 1.22765469e+13 1.57877404e+14 7.30913905e+14
 1.21818984e+15 3.38386067e+15 9.47264421e+13 9.47264421e+13
 2.63129006e+14 3.41015192e+13 1.57877404e+14 1.21818984e+15
 1.21818984e+15 2.63129006e+14 4.38548343e+14 2.65173413e+12
 5.68358653e+13 1.21818984e+15 5.68358653e+13 7.30913905e+14
 3.41015192e+13 1.57877404e+14 1.21818984e+15 1.57877404e+14
 7.36592814e+12 4.41955688e+12 4.38548343e+14 4.38548343e+14
 4.38548343e+14 4.35167268e+16 7.30913905e+14 5.68358653e+13
 3.4101

Randomized Voting Training

In [11]:
rmv = random_voter.make_classifiers(train, trainc, features, which_gain, 100, 200, 2, maxlevel=5)
out = random_voter.randomized_vote(rmv, test, testc)

#calculate accuracy
a = np.zeros(len(out))
b = np.zeros(len(out))
d = np.zeros(len(out))

for i in range(len(out)):
    if testc[i] in ['good', 'v-good']:
        b[i] = 1
        if out[i] == testc[i]:
            d[i] = 1
    if out[i] == testc[i]:
        a[i] = 1
    
print("Randomized Majority Voter Training")
print("Number correctly predicted", np.sum(a))
print("Number of test points", len(a))
print("Percentage Accuracy", np.sum(a) / len(a) * 100.0)

Updated weights: [2.06198846e+11 7.42315846e+10 1.23719308e+11 9.54624287e+11
 2.06198846e+11 2.67233704e+10 4.41955688e+12 5.72774572e+11
 5.77224802e+09 7.42315846e+10 4.45389507e+10 5.72774572e+11
 1.23719308e+11 4.45389507e+10 4.45389507e+10 5.77224802e+09
 2.06198846e+11 9.62041336e+09 9.62041336e+09 7.42315846e+10
 2.65173413e+12 2.06198846e+11 4.45389507e+10 9.62041336e+09
 7.48083343e+08 2.67233704e+10 9.54624287e+11 7.42315846e+10
 9.54624287e+11 7.42315846e+10 3.43664743e+11 7.42315846e+10
 1.23719308e+11 4.45389507e+10 2.06198846e+11 5.72774572e+11
 9.62041336e+09 1.24680557e+09 1.60340223e+10 3.43664743e+11
 4.45389507e+10 4.45389507e+10 2.07800929e+09 7.42315846e+10
 7.42315846e+10 2.67233704e+10 2.65173413e+12 4.45389507e+10
 9.54624287e+11 5.77224802e+09 5.77224802e+09 1.59104048e+12
 2.65173413e+12 1.23719308e+11 9.54624287e+11 3.46334881e+09
 3.43664743e+11 4.48850006e+08 1.60340223e+10 3.43664743e+11
 3.43664743e+11 5.68358653e+13 1.59104048e+12 2.65173413e+12
 5.7277